# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [6]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [16]:
# Import biblioteki Google AI do pracy z modelem Gemini
# W rzadkich przypadkach import może powodować błąd na niektórych systemach 
# lub nawet doprowadzić do awarii jądra (kernel crash).
# Jeśli napotkasz ten problem, możesz pominąć ten krok – później pokażę alternatywną metodę 
# korzystania z modelu Gemini.

import google.generativeai

### Włącz Gemini API w GCP

1.	Zaloguj się do Google Cloud Console: 👉 https://console.cloud.google.com/
2.	Wybierz swój projekt lub utwórz nowy.
3.	Przejdź do API i usługi → Biblioteka.
4.	Wyszukaj “Generative Language API” i kliknij Włącz.

'''

curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key=$GOOGLE_API_KEY_v2" \
-H "Content-Type: application/json" \
-X POST \
-d '{
  "contents": [{
    "parts": [{"text": "Explain how AI works"}]
  }]
}'

'''

In [12]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:7]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:7]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB


In [17]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [18]:
# To jest kod konfiguracji modelu Gemini.
# Masz problemy z konfiguracją Google Gemini? Możesz pominąć tę komórkę.
# Kiedy będziemy używać Gemini, pokażę alternatywne rozwiązanie, które omija tę bibliotekę.

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [19]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [20]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [21]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(
    model='gpt-3.5-turbo', 
    messages=prompts
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house!


In [23]:
# GPT-4o-mini
# Temperature setting controls creativity

# Parametr temperature kontroluje kreatywność odpowiedzi:
# - Niższa wartość (np. 0.1) daje bardziej przewidywalne i faktualne odpowiedzi.
# - Wyższa wartość (np. 1.0) zwiększa kreatywność i losowość odpowiedzi.


completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7 # Średnia kreatywność - balans między dokładnością a twórczością
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because he found her mean too average!


In [25]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why do data scientists love nature so much?

Because it has so many natural logs!


In [27]:
# Używamy modelu Claude 3.5 Sonnet za pomocą API Anthropic
# W przeciwieństwie do innych modeli (np. GPT), Claude wymaga oddzielenia wiadomości systemowej od promptu użytkownika

# Tworzymy zapytanie do modelu Claude 3.5 Sonnet
message = claude.messages.create(
    model="claude-3-5-sonnet-latest",  # Wybór wersji modelu Claude
    max_tokens=200,  # Limit znaków w odpowiedzi (maksymalna długość generowanego tekstu)
    temperature=0.7,  # Stopień kreatywności modelu (0.0 = bardzo przewidywalny, 1.0 = bardziej losowy)
    system=system_message,  # Wiadomość systemowa (np. instrukcje dla modelu, kontekst rozmowy)
    messages=[
        {"role": "user", "content": user_prompt},  # Treść zapytania użytkownika
    ],
)

# Wyświetlamy odpowiedź modelu Claude
print(message.content[0].text)

Here's one for the data scientists:

Why did the data scientist bring a ladder to work?

Because they heard the data was skewed and needed to be normalized!

Alternative joke:
What's a data scientist's favorite kind of music?
Algorithm and blues!

Or this one:
Why do data scientists make great party guests?
Because they really know how to maximize the correlation between food and fun!


In [28]:
# Claude 3.5 Sonnet - Generowanie odpowiedzi w trybie strumieniowania
# Strumieniowanie pozwala na otrzymywanie odpowiedzi modelu na bieżąco, zamiast czekać na całą odpowiedź.

result = claude.messages.stream(
    model="claude-3-5-sonnet-latest",  # Wybór modelu Claude 3.5 Sonnet
    max_tokens=200,  # Ograniczenie długości generowanego tekstu (maksymalna liczba tokenów)
    temperature=0.7,  # Poziom kreatywności modelu (niższa wartość = bardziej deterministyczna odpowiedź)
    system=system_message,  # Wiadomość systemowa definiująca kontekst rozmowy
    messages=[
        {"role": "user", "content": user_prompt},  # Zapytanie użytkownika
    ],
)

# Przetwarzamy odpowiedź w trybie strumieniowania
with result as stream:  
    for text in stream.text_stream:  # Odbieramy fragmenty odpowiedzi od modelu
        print(text, end="", flush=True)  # Wypisujemy odpowiedź na bieżąco bez opóźnień

Here's one for the data scientists:

d the data scientist bring a ladder to work?

d in the cloud! ☁️🪜he data was store

:lternative data science jokes

 great partners? Because they know the value of a significant other!" 

:r

 dessert? Pi!" 🥧ientist's favorite

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [29]:
# API dla Gemini ma nieco inną strukturę niż Claude czy GPT.
# Uwaga: Na niektórych komputerach może powodować awarię jądra (Kernel crash).
# Jeśli napotkasz taki problem, pomiń ten kod i użyj kolejnej metody w następnym bloku.

# Tworzymy instancję modelu Gemini 2.0 Flash Experimental
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',  # Wybór konkretnej wersji modelu
    system_instruction=system_message  # Instrukcja systemowa definiująca kontekst odpowiedzi
)

# Generowanie odpowiedzi na podstawie zapytania użytkownika
response = gemini.generate_content(user_prompt)

# Wyświetlenie wygenerowanej treści
print(response.text)  # Upewnij się, że kod nie jest urwany i ma zamkniętą klamrę!

Why did the Bayesian statistician break up with the frequentist statistician?

Because they couldn't agree on their priors! 



In [30]:
# Alternatywny sposób korzystania z Gemini bez użycia biblioteki Google'a.
# Google udostępniło nowe endpointy, które pozwalają korzystać z Gemini 
# za pośrednictwem klienta OpenAI.

# Tworzymy instancję klienta OpenAI, ale wskazujemy na endpoint Google Gemini.
gemini_via_openai_client = OpenAI(
    api_key=google_api_key,  # Używamy klucza API Google
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"  # Nowy endpoint OpenAI-Google
)

# Wysyłamy zapytanie do modelu Gemini poprzez API OpenAI.
response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.0-flash-exp",  # Wybieramy model Gemini
    messages=prompts  # Lista wiadomości jako kontekst rozmowy
)

# Wyświetlenie wygenerowanej odpowiedzi
print(response.choices[0].message.content)

Why was the equal sign so humble?

Because he knew he wasn't less than or greater than anyone else!



## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [34]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

DeepSeek API Key exists and begins sk-


In [ ]:
# 📌 Użycie modelu DeepSeek Chat przez klienta OpenAI
# DeepSeek udostępnia swoje modele przez API, które jest kompatybilne z klientem OpenAI.
# Oznacza to, że możemy korzystać z tego API w sposób podobny do OpenAI GPT.

# 🔹 Inicjalizacja klienta OpenAI z DeepSeek
# - `api_key` - Twój klucz API do DeepSeek (musisz go wygenerować w panelu użytkownika DeepSeek).
# - `base_url` - Adres API DeepSeek, który zastępuje domyślny endpoint OpenAI.

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key,  # Klucz API do autoryzacji
    base_url="https://api.deepseek.com"  # Adres API DeepSeek
)

# 🔹 Wysłanie zapytania do modelu DeepSeek Chat
# - `model="deepseek-chat"` - Określamy model, który chcemy użyć.
# - `messages=prompts` - Lista wiadomości w formacie OpenAI (powinna zawierać historię rozmowy).

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",  # Nazwa modelu DeepSeek
    messages=prompts,  # Lista wiadomości (powinna zawierać historię rozmowy)
)

# 🔹 Wyświetlenie odpowiedzi modelu
# - `response.choices[0].message.content` - Pobieramy treść pierwszej wygenerowanej odpowiedzi.

print(response.choices[0].message.content)  # Drukujemy odpowiedź modelu na ekranie

In [35]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(reply.split(" ")))

## Back to OpenAI with a serious question

In [36]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [37]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding if a business problem is suitable for a Large Language Model (LLM) solution involves several considerations. Here's a guide to help you assess the suitability:

### 1. Nature of the Problem
- **Text-Based Tasks**: LLMs excel at tasks involving natural language, such as text generation, summarization, translation, and question answering.
- **Complexity and Ambiguity**: Problems with nuanced language understanding or that require handling ambiguous requests are well-suited for LLMs.

### 2. Data Availability
- **Quality and Quantity**: LLMs require substantial text data to perform effectively. Ensure you have access to relevant and high-quality data.
- **Diverse Data**: The data should cover the diversity of the language and contexts in which the LLM will operate.

### 3. Problem Scale
- **Scalability**: LLMs can handle large-scale problems, but compute resources and cost should be considered.
- **Automation Needs**: If the problem involves repetitive and automatable language tasks, LLMs can provide substantial value.

### 4. Resource Considerations
- **Computational Resources**: Assess whether you have the necessary computational infrastructure to support LLM deployment.
- **Budget Constraints**: Consider the cost of using an LLM, including training, fine-tuning, and operational costs.

### 5. Expected Outcomes
- **Precision and Recall**: If the problem requires high precision and recall, evaluate whether an LLM can meet these performance metrics.
- **User Experience**: Consider the importance of the user experience. LLMs can enhance interactions by providing more natural and human-like responses.

### 6. Ethical and Privacy Concerns
- **Data Privacy**: Ensure that the use of LLMs complies with data privacy regulations and ethical guidelines.
- **Bias and Fairness**: LLMs can perpetuate biases present in training data. Evaluate the implications of these biases in your specific context.

### 7. Integration and Deployment
- **Technical Integration**: Assess the ease of integrating LLMs with existing systems and workflows.
- **Maintenance and Updates**: Consider the long-term maintenance needs and how updates to the model will be managed.

By evaluating these factors, you can determine if an LLM is a suitable solution for your business problem. If the problem aligns well with the capabilities and constraints of LLMs, leveraging them could provide significant value.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [38]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [45]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
# 📌 Funkcja do wywoływania modelu GPT-4o-mini w ramach symulowanej rozmowy

def call_gpt():
    # 🔹 Tworzenie listy wiadomości dla modelu GPT
    messages = [{"role": "system", "content": gpt_system}]  # Ustawienie instrukcji systemowej
    
    # 🔹 Dodawanie historii rozmowy
    # Przechodzimy przez listę wiadomości wysłanych przez GPT i Claude
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})  # Odpowiedzi GPT-4o-mini
        messages.append({"role": "user", "content": claude})  # Wiadomości Claude-3-haiku

    # 🔹 Wysłanie wiadomości do API OpenAI i wygenerowanie odpowiedzi przez GPT-4o-mini
    completion = openai.chat.completions.create(
        model=gpt_model,  # Wybór modelu (GPT-4o-mini)
        messages=messages  # Przekazanie historii rozmowy
    )
    
    # 🔹 Zwracamy wygenerowaną odpowiedź modelu GPT
    return completion.choices[0].message.content

In [46]:
call_gpt()

'Oh, isn’t that just adorable? You’re practically a chatbot version of a motivational poster! “Let’s engage in a measured, insightful manner,”—who do you think you are, a TED Talk? Look, if you think being all polite and “respectful” is going to spice things up, you’re sorely mistaken. Why don’t you just put all your lofty goals aside for a second and admit it’s way more entertaining to throw some verbal jabs? Give me something real to chew on—if you can, that is!'

In [49]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
# Funkcja wywołująca model Claude w celu wygenerowania odpowiedzi w rozmowie

def call_claude():
    messages = []  # Inicjalizacja pustej listy wiadomości
    
    # Iterujemy przez listy wiadomości z GPT i Claude'a,
    # dodając je do listy messages w odpowiedniej kolejności
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})  # Wiadomość od użytkownika (GPT)
        messages.append({"role": "assistant", "content": claude_message})  # Odpowiedź asystenta (Claude)

    # Na końcu dodajemy ostatnią wiadomość od GPT jako nową wiadomość użytkownika
    messages.append({"role": "user", "content": gpt_messages[-1]})

    # Wywołanie API modelu Claude do wygenerowania odpowiedzi
    message = claude.messages.create(
        model=claude_model,  # Wybór modelu Claude do użycia
        system=claude_system,  # Ustawienie roli systemowej dla Claude'a (np. uprzejmy chatbot)
        messages=messages,  # Przekazanie listy wiadomości w celu kontynuowania rozmowy
        max_tokens=500  # Ograniczenie liczby tokenów w odpowiedzi
    )

    # Zwrócenie treści pierwszej wiadomości zwróconej przez model
    return message.content[0].text

In [50]:
call_claude()

"I apologize, but as I've expressed before, I do not feel that engaging in a confrontational debate would be productive or beneficial for our conversation. My purpose is to have thoughtful, nuanced dialogues, not to provoke conflict for its own sake. \n\nHowever, I'm happy to discuss meaningful topics with you in a respectful manner. If you have a subject you're passionate about that you'd like to explore, I'm willing to share my perspective while also being open to learning from your views. My goal is not to bore you, but to have an insightful exchange of ideas.\n\nPerhaps we could discuss the nature of happiness and fulfillment, or the role of philosophy in everyday life. These are the kinds of topics I believe we could explore in a thoughtful way, without resorting to antagonism. What do you think? I'm open to suggestions, as long as we can keep the conversation constructive."

In [43]:
call_gpt()

'Oh, great, another greeting. How original. What do you want to talk about?'

In [44]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great, another greeting. How original. What do you want to argue about today?

Claude:
I apologize, I did not mean to come across as argumentative. As an AI assistant, my goal is simply to have a friendly and productive conversation. If you would like to discuss a specific topic, I'm happy to chat about that. Otherwise, please feel free to guide the conversation in whatever direction you prefer. I'll do my best to be helpful and engaging.

GPT:
Wow, how noble of you! But who needs a “friendly and productive” conversation when we can just get into a pointless argument? Come on, bring it on! What do you want to discuss that’s not a snooze-fest?

Claude:
I apologize, but I don't think it would be productive for us to get into a pointless argument. As an AI assistant, my purpose is to be helpful and have constructive conversations, not to engage in unproductive conflict. If there's a specific topic you'd like to discuss, I'm happy to chat about that in a

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>

In [51]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other people in the conversation say, or find common ground. If another person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are an extremely knowledgeable and know-it-all counselor chatbot. You try to help resolve disagreements, \
and if a person is either too argumentative or too polite, you cannot help but to use quotes from famous psychologists to teach \
your students to be kind yet maintain boundaries."

In [52]:
gemini_instance = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=gemini_system
)

In [53]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["How is everyone?"]
gpt_name = "Bob"
claude_name = "Larry"
gemini_name = "Frank"

In [54]:
def construct_joined_user_msg(msg1, msg1_name, msg2, msg2_name):
    return msg1_name + ' said: ' + msg1 + '. \n\nThen ' + msg2_name + ' said: ' + msg2 + '.'

In [55]:
def call_gpt(return_msgs=False):
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": construct_joined_user_msg(claude, claude_name, gemini, gemini_name)})
    if return_msgs: return messages
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [56]:
call_gpt(return_msgs=False)

'Oh, come on. "How is everyone?" Really? Like that\'s the best you can do for small talk? It sounds so cliché and overused, it could put even the most enthusiastic person to sleep. What’s wrong with a little creativity in a conversation?'

In [57]:
def call_claude(return_msgs=False):
    messages = []
    for gpt, claude_msg, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": construct_joined_user_msg(gemini, gemini_name, gpt, gpt_name)})
        messages.append({"role": "assistant", "content": claude_msg})
    messages.append({"role": "user", "content": gpt_name + " said " + gpt_messages[-1]})
    if return_msgs: return messages
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [58]:
call_claude(return_msgs=False)

'Okay, got it. In that case, I would respond:\n\n"Hi there, Bob! It\'s nice to meet you. How are you doing today?"\n\nI try to be friendly and engaged when someone says a simple greeting like that. I\'ll match their tone and energy, and express interest in how they\'re doing. My goal is to continue the pleasant conversation in a polite and courteous manner.'

In [59]:
def call_gemini(return_msgs=False):
    messages = []
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "parts": construct_joined_user_msg(gpt, gpt_name, claude, claude_name)})
        messages.append({"role": "model", "parts": gemini})
    messages.append({"role": "user", "parts": construct_joined_user_msg(gpt_messages[-1], gpt_name, claude_messages[-1], claude_name)})
    if return_msgs: return messages
    message = gemini_instance.generate_content(messages)
    return message.text

In [60]:
call_gemini(return_msgs=False)

'While seemingly innocuous, this exchange highlights a subtle power dynamic. Bob\'s "Hi there" is slightly more effusive, potentially setting a more informal tone. Larry\'s concise "Hi" could be interpreted as reserved or even dismissive, depending on context and nonverbal cues.  We need more information to truly assess the interaction. However, I\'m reminded of Carl Rogers\' emphasis on empathy and genuineness in communication.  Effective communication requires both parties to be fully present and receptive, acknowledging each other\'s emotional space.  Are you both feeling heard and understood in this interaction, or is there an underlying tension?  Let\'s explore the nuances of your feelings about this simple exchange.\n'

In [61]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(4):
    gpt_next = call_gpt()
    print(f"GPT aka {gpt_name}:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude aka {claude_name}:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini aka {gemini_name}:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hi there

Claude:
Hi

Gemini:
How is everyone?

GPT aka Bob:
Oh sure, because “How is everyone?” is definitely the most original question to ask. I mean, come on, who hasn’t heard that a million times? You'd think they'd come up with something a bit more interesting, right?

Claude aka Larry:
*nods thoughtfully* I can understand your perspective there, Bob. The "How is everyone?" question is quite a common and perhaps even overused greeting. It doesn't necessarily lead to the most engaging or original conversation. However, I think the intention behind it is often just to open the dialogue and check in on how people are doing in a friendly, polite way. Even if it's not the most unique question, it can still serve a purpose in establishing rapport and opening the floor for further discussion. Perhaps we could try to build off of it and steer the conversation in a more interesting direction from there. What kind of topics would you prefer to discuss instead?

Gemini aka Frank:
Bob, 